In [ ]:
import pandas

# Load the data
data = pandas.read_csv('/mnt/data/modified_data.csv')

# Convert TimeStamp to datetime and string format for easier handling
data['TimeStamp'] = pandas.to_datetime(data['TimeStamp'], format='%Y%m%d%H%M%S')

# Convert speed from miles per hour to kilometers per hour
data['Speed'] = data['Speed'] * 1.60934

# Calculate time differences and accumulated time for trips
data.sort_values(by=['TripId', 'TimeStamp'], inplace=True)
data['TimeDelta'] = data.groupby('TripId')['TimeStamp'].diff().fillna(pandas.Timedelta(seconds=0)).dt.total_seconds()
data['AccumulatedTime'] = data.groupby('TripId')['TimeDelta'].cumsum()

# Add a column for movement direction based on Heading changes
data['DirectionChange'] = data.groupby('TripId')['Heading'].diff().fillna(0).abs()
data['Movement'] = data['DirectionChange'].apply(lambda x: 'Straight' if x < 10 else 'Turning')

# Example plot of speed over time for trip T1
trip_data = data[data['TripId'] == 'T1']
plot = trip_data.plot(x='TimeStamp', y='Speed', title='Speed over Time for Trip T1', xlabel='Time', ylabel='Speed (km/h)')
print('Plotting complete.')

# Display basic statistics
print('Basic Speed Statistics:\n', data['Speed'].describe())
print('\nAccumulated Driving Time for Each Trip:\n', data.groupby('TripId')['AccumulatedTime'].max())